<a href="https://colab.research.google.com/github/waltherwj/Tensorflow2-lectures/blob/master/tf2_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>